In [1]:
import mnist
import scipy.misc
import numpy as np
from gurobipy import *

train_images = mnist.train_images()
train_labels = mnist.train_labels()

samples = train_images.reshape((train_images.shape[0], train_images.shape[1] * train_images.shape[2]))

#this is called one-hot-encoding if transposed, like this we generate 10 vectors of 60k 0-1s per type
labels = np.zeros((10,60000), dtype=int)
for i in range(len(train_labels)):
    labels[train_labels[i],i] = 1


In [10]:
#Modelparameters: N Instances, k Features, s=m1 H1-Neurons, t=m2 H2-Neurons
def build_model(N,n,m1,m2,samples,labels):
    m = Model()
    M = 100000

    #Modeloutput
    y_hat = m.addVars(N, name="output",vtype=GRB.BINARY)
    
    #Neuronoutput
    a1 = m.addVars(N,m1, name="a1", vtype=GRB.BINARY)
    a2 = m.addVars(N,m2, name="a2", vtype=GRB.BINARY) 
    
    #Bias
    b1 = m.addVars(m1, lb=-1, ub=1, name="bias1",vtype=GRB.INTEGER)
    b2 = m.addVars(m2, lb=-1, ub=1, name="bias2",vtype=GRB.INTEGER)
    b3 = m.addVars(1, lb=-1, ub=1, name="bias3",vtype=GRB.INTEGER)
    
    #Weightmatrices
    w1 = m.addVars(n,m1, lb=-1, ub=1, name="w1",vtype=GRB.INTEGER)
    w2 = m.addVars(m1,m2, lb=-1, ub=1, name="w2",vtype=GRB.INTEGER)
    w3 = m.addVars(m2, lb=-1, ub=1, name="w3",vtype=GRB.INTEGER)
    
    #Treshholds
    theta1 = m.addVars(m1, lb=-M/2, ub=M/2, name="theta1",vtype=GRB.INTEGER)
    theta2 = m.addVars(m2, lb=-M/2, ub=M/2, name="theta2",vtype=GRB.INTEGER)
    theta3 = m.addVars(1, lb=-M/2, ub=M/2, name="theta3",vtype=GRB.INTEGER)
    
    
    #Auxillary
    zeta1 = m.addVars(N,m1,m2, name="zeta1",vtype=GRB.INTEGER)
    zeta2 = m.addVars(N,m2, name="zeta2",vtype=GRB.INTEGER)
    
    m.update()
    print("ready")

    ############   minimize
    obj = 1/N * sum((y_hat[i] - labels[i])*(y_hat[i] - labels[i]) for i in range(N))
    m.setObjective(obj,GRB.MINIMIZE)

    ############   subject to
    
    
    #Layer1
    m.addConstrs(quicksum(samples[i,k]*w1[k,s] for k in range(n)) + b1[s] >= theta1[s] - M*(1-a1[i,s]) for s in range(m1) for i in range(N))
    m.addConstrs(quicksum(samples[i,k]*w1[k,s] for k in range(n)) + b1[s] <= theta1[s]-1 + M*a1[i,s] for s in range(m1) for i in range(N))

    #Layer2
    m.addConstrs(quicksum(zeta1[i,s,t] for s in range(m1)) + b2[t] >= theta2[t] - M*(1-a2[i,t]) for t in range(m2) for i in range(N))
    m.addConstrs(quicksum(zeta1[i,s,t] for s in range(m1)) + b2[t] <= theta2[t]-1 + M*a2[i,t] for t in range(m2) for i in range(N))
    
    m.addConstrs(zeta1[i,s,t] >= w2[s,t] - M*(1-a1[i,s]) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta1[i,s,t] <= w2[s,t] + M*(1-a1[i,s]) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta1[i,s,t] >= - M*a1[i,s] for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta1[i,s,t] <= M*a1[i,s] for s in range(m1) for t in range(m2) for i in range(N))
    
    #Layer3
    m.addConstrs(quicksum(zeta2[i,t] for t in range(m2)) + b3[0] >= theta3[0] - M*(1-y_hat[i]) for i in range(N))
    m.addConstrs(quicksum(zeta2[i,t] for t in range(m2)) + b3[0] <= theta3[0]-1 + M*y_hat[i] for i in range(N))
    
    m.addConstrs(zeta2[i,t] >= w3[t] - M*(1-a2[i,t]) for t in range(m2) for i in range(N))
    m.addConstrs(zeta2[i,t] <= w3[t] + M*(1-a2[i,t]) for t in range(m2) for i in range(N))
    m.addConstrs(zeta2[i,t] >= - M*a2[i,t] for t in range(m2) for i in range(N))
    m.addConstrs(zeta2[i,t] <= M*a2[i,t] for t in range(m2) for i in range(N))
    
    
    m.update()
    
    m.Params.outputFlag = 1
    m.Params.timelimit = 18000.0 # 5Stunden maximale Rechenzeit
    
    print("Model finalized, return Model.")
    return m, w1, w2, w3, theta1, theta2, theta3 

In [11]:
#choose class i to optimize for in 1 vs all strategy
def solve_model(N,n,m1,m2,samples,labels,i):
    m, w1, w2, w3, theta1, theta2, theta3 = build_model(N,n,m1,m2,samples,labels[i])

    print("Optimize given Model.")
    m.optimize()


    #create result dicts for 1 vs all Model m:
    dic_w1 = m.getAttr('x', w1)
    dic_w2 = m.getAttr('x', w2)
    dic_w3 = m.getAttr('x', w3)

    dic_theta1 = m.getAttr('x', theta1)
    dic_theta2 = m.getAttr('x', theta2)
    dic_theta3 = m.getAttr('x', theta3)
    
    mydict = {
                "w1": dic_w1, 
                "w2": dic_w2, 
                "w3": dic_w3, 
                "theta1": dic_theta1, 
                "theta2": dic_theta2, 
                "theta3": dic_theta3
    }
    
    return mydict

In [ ]:
N= 1000 #500 or 1000 or 5000 otherwise no existing folder
for i in range(10):
    with open('results/Integer_NN/'+str(N)+'/Architecture_Class_'+str(i)+'.txt', 'w') as f: 
        print(solve_model(N,samples.shape[1],10,10,samples,labels,i), file=f)

ready
Parameter outputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter timelimit to 18000.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Model finalized, return Model.
Optimize given Model.
Optimize a model with 842000 rows, 138992 columns and 5314400 nonzeros
Model has 1000 quadratic objective terms
Variable types: 0 continuous, 138992 integer (21000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-03, 2e-03]
  QObjective range [2e-03, 2e-03]
  Bounds range     [1e+00, 5e+04]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 0.0970000
Presolve removed 470000 rows and 1771 columns (presolve time = 5s) ...
Presolve removed 470000 rows and 1771 columns (presolve time = 11s) ...
Presolve removed 470000 rows and 1771 columns (presolve time = 15s) ...
Presolve removed 470000 rows and 1771 columns
Presolve time: 18.91s
Presolved: 372000 rows, 137221 columns, 4042400 nonzeros
Variable types

Model has 1000 quadratic objective terms
Variable types: 0 continuous, 138992 integer (21000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [2e-03, 2e-03]
  QObjective range [2e-03, 2e-03]
  Bounds range     [1e+00, 5e+04]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 0.1160000
Presolve removed 470000 rows and 1771 columns (presolve time = 5s) ...
Presolve removed 470000 rows and 1771 columns (presolve time = 10s) ...
Presolve removed 470000 rows and 1771 columns (presolve time = 15s) ...
Presolve removed 470000 rows and 1771 columns
Presolve time: 18.42s
Presolved: 372000 rows, 137221 columns, 4042400 nonzeros
Variable types: 0 continuous, 137221 integer (131110 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -8.3266727e-17   7.632328e+05   0.000000e+00     20s
    7852    2.3199536e-06   0.000000e+00   0.000000e+00     20s

Root relaxation: objective 2.319954e-06, 78

In [17]:
with open('myfile.txt', 'w') as f:
    print(classifier, file=f)

10


In [30]:
# Make Validation samples
import mnist
import numpy as np

test_images = mnist.test_images()
test_labels = mnist.test_labels()

testsamples = test_images.reshape(
    (test_images.shape[0], test_images.shape[1] * test_images.shape[2])
)

testlabels = np.zeros((10000,10), dtype=int)
for i in range(len(test_labels)):
    testlabels[i,test_labels[i]] = 1


mydict = {}
for i in range(10):
    count = 0
    for n in range(len(testsamples)):
        if testlabels[n,i] == 1:
            count +=1
    mydict[str(i)] = count
print(mydict)

{'0': 980, '1': 1135, '2': 1032, '3': 1010, '4': 982, '5': 892, '6': 958, '7': 1028, '8': 974, '9': 1009}


In [ ]:
def cross_valid(classifier,testlabels, testsamples):
    count = 0
    for n in range(len(testsamples)):
        zeta1=np.zeros(r_h1neurons)
        for r in range(r_h1neurons):
            test1 = sum(CVtest[n,m] * dic_w1[m,r] for m in range(len(CVtest[0])))

            if test1 >= dic_theta1[r]:
                zeta1[r] = 1

        zeta2 = np.zeros(s_h2neurons)
        for s in range(s_h2neurons): 
            test2 = sum(zeta1[r]*dic_w2[r,s] for r in range(r_h1neurons))

            if test2 >= dic_theta2[s]:
                zeta2[s] = 1

        test3 = sum(zeta2[s]*dic_w3[s] for s in range(s_h2neurons))

        if test3 >= dic_theta3[0]:
            out = 1
        else:
            out = -1

        if out == CVtestL[n]:
            count += 1
    return count/len(CVtestL)*100

## Vektormodell follows

In [2]:
#Modelparameters: N Instances, k Features, s=m1 H1-Neurons, t=m2 H2-Neurons
def build_model_vectorout(N,n,m1,m2,samples,labels):
    m = Model()
    M = 5000
    
    m3 = 10

    #Modeloutput
    y_hat = m.addVars(N,m3, name="output",vtype=GRB.BINARY)#
    
    #Neuronoutput
    a1 = m.addVars(N,m1, name="a1", vtype=GRB.BINARY)
    a2 = m.addVars(N,m2, name="a2", vtype=GRB.BINARY) 
    
    #Bias
    b1 = m.addVars(m1, name="bias1",vtype=GRB.INTEGER)
    b2 = m.addVars(m2, name="bias2",vtype=GRB.INTEGER)
    b3 = m.addVars(m3, name="bias3",vtype=GRB.INTEGER)#
    
    #Weightmatrices
    w1 = m.addVars(n,m1, name="w1",vtype=GRB.INTEGER)
    w2 = m.addVars(m1,m2, name="w2",vtype=GRB.INTEGER)
    w3 = m.addVars(m2,m3, name="w3",vtype=GRB.INTEGER)#
    
    #Treshholds
    theta1 = m.addVars(m1, name="theta1",vtype=GRB.INTEGER)
    theta2 = m.addVars(m2, name="theta2",vtype=GRB.INTEGER)
    theta3 = m.addVars(m3, name="theta3",vtype=GRB.INTEGER)#
    
    
    #Auxillary
    zeta1 = m.addVars(N,m1,m2, name="zeta1",vtype=GRB.INTEGER)
    zeta2 = m.addVars(N,m2,m3, name="zeta2",vtype=GRB.INTEGER)#
    
    m.update()
    print("ready")

    ############   minimize
    obj = 1/N * sum(sum((y_hat[i,l] - labels[l,i])*(y_hat[i,l] - labels[l,i]) for l in range(m3)) for i in range(N))
    m.setObjective(obj,GRB.MINIMIZE)

    ############   subject to
    
    
    #Layer1
    m.addConstrs(quicksum(samples[i,k]*w1[k,s] for k in range(n)) + b1[s] >= theta1[s] - M*(1-a1[i,s]) for s in range(m1) for i in range(N))
    m.addConstrs(quicksum(samples[i,k]*w1[k,s] for k in range(n)) + b1[s] <= theta1[s]-1 + M*a1[i,s] for s in range(m1) for i in range(N))

    #Layer2
    m.addConstrs(quicksum(zeta1[i,s,t] for s in range(m1)) + b2[t] >= theta2[t] - M*(1-a2[i,t]) for t in range(m2) for i in range(N))
    m.addConstrs(quicksum(zeta1[i,s,t] for s in range(m1)) + b2[t] <= theta2[t]-1 + M*a2[i,t] for t in range(m2) for i in range(N))
    
    m.addConstrs(zeta1[i,s,t] >= w2[s,t] - M*(1-a1[i,s]) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta1[i,s,t] <= w2[s,t] + M*(1-a1[i,s]) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta1[i,s,t] >= - M*a1[i,s] for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta1[i,s,t] <= M*a1[i,s] for s in range(m1) for t in range(m2) for i in range(N))
    
    #Layer3
    m.addConstrs(quicksum(zeta2[i,t,l] for t in range(m2)) + b3[l] >= theta3[l] - M*(1-y_hat[i,l]) for l in range(m3) for i in range(N))
    m.addConstrs(quicksum(zeta2[i,t,l] for t in range(m2)) + b3[l] <= theta3[l]-1 + M*y_hat[i,l] for l in range(m3) for i in range(N))
    
    m.addConstrs(zeta2[i,t,l] >= w3[t,l] - M*(1-a2[i,t]) for l in range(m3) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta2[i,t,l] <= w3[t,l] + M*(1-a2[i,t]) for l in range(m3) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta2[i,t,l] >= - M*a2[i,t] for l in range(m3) for s in range(m1) for t in range(m2) for i in range(N))
    m.addConstrs(zeta2[i,t,l] <= M*a2[i,t] for l in range(m3) for s in range(m1) for t in range(m2) for i in range(N))
    
    #Constrains
    m.addConstrs(b1[s] >= -1 for s in range(m1))
    m.addConstrs(b1[s] <= 1 for s in range(m1))
    m.addConstrs(b2[t] >= -1 for t in range(m2))
    m.addConstrs(b2[t] <= 1 for t in range(m2))
    m.addConstrs(b3[l] >= -1 for l in range(m3)) 
    m.addConstrs(b3[l] <= 1 for l in range(m3))
    
    
    m.addConstrs(w1[k,s] >= -1 for k in range(n) for s in range(m1))
    m.addConstrs(w1[k,s] <= 1 for k in range(n) for s in range(m1))
    m.addConstrs(w2[s,t] >= -1 for s in range(m1) for t in range(m2))
    m.addConstrs(w2[s,t] <= 1 for s in range(m1) for t in range(m2))
    m.addConstrs(w3[t,l] >= -1 for t in range(m2) for l in range(m3))
    m.addConstrs(w3[t,l] <= 1 for t in range(m2) for l in range(m3))
    
    
    m.update()
    
    print("Model finalized, return Model.")
    return m

In [ ]:
#optimize over all classes simultanously
m_vec = build_model_vectorout(100,samples.shape[1],10,10,samples,labels)

print("Optimize given Model.")
m_vec.optimize()

Academic license - for non-commercial use only
ready
Model finalized, return Model.
Optimize given Model.
Optimize a model with 462140 rows, 31100 columns and 1466660 nonzeros
Model has 1000 quadratic objective terms
Variable types: 0 continuous, 31100 integer (3000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [2e-02, 2e-02]
  QObjective range [2e-02, 2e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 9.0000000
Found heuristic solution: objective 1.0000000
Presolve removed 396140 rows and 2880 columns
Presolve time: 3.56s
Presolved: 66000 rows, 28220 columns, 484480 nonzeros
Variable types: 0 continuous, 28220 integer (28180 binary)

Root relaxation: objective 2.000000e-04, 2635 iterations, 0.14 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00020    0  100  

H 6778  5466                       0.3000000    0.00020   100%   554 1280s
  6782  5468    0.15000   24  419    0.30000    0.00020   100%   554 1285s
  6803  5478    0.15000   25  411    0.30000    0.00020   100%   553 1292s
  6831  5497    0.15000   27  770    0.30000    0.00020   100%   551 1297s
H 6854  5247                       0.2800000    0.00020   100%   552 1328s
H 6856  4992                       0.2700000    0.00020   100%   552 1328s
  6860  4993    0.15000   29  555    0.27000    0.00020   100%   552 1334s
  6885  5002    0.15000   30  563    0.27000    0.00020   100%   552 1340s
  6908  5012    0.15000   32  847    0.27000    0.00020   100%   552 1351s
  6926  5029    0.15000   33  830    0.27000    0.00020   100%   552 1358s
  6954  5048    0.15000   35  839    0.27000    0.00020   100%   551 1363s
  6990  5065    0.15000   36  761    0.27000    0.00020   100%   550 1368s
  7023  5094    0.15000   38  676    0.27000    0.00020   100%   550 1373s
  7062  5120    0.15000  

 33689 24792    0.00020   76 4090    0.21000    0.00020   100%  1015 11720s
 34349 25431    0.11000  254 3242    0.21000    0.00020   100%  1025 11932s
 35162 26164    0.00020  185 1851    0.21000    0.00020   100%  1018 12347s
 35632 26656    0.00020  272 1958    0.21000    0.00020   100%  1026 12580s
 36188 27163    0.00020  350 2008    0.21000    0.00020   100%  1040 12801s
 36999 27882    0.00020  142 3609    0.21000    0.00020   100%  1049 12999s
 37633 28461    0.00020   89 4248    0.21000    0.00020   100%  1059 13209s
 38403 29212    0.09955  250 5980    0.21000    0.00020   100%  1063 13474s
 39085 29837    0.10864  285 5246    0.21000    0.00020   100%  1063 13693s
 39408 30154    0.18705  322 6121    0.21000    0.00020   100%  1075 13933s
 39839 30555    0.18750  325 6132    0.21000    0.00020   100%  1090 14445s
 39886 30612     cutoff  326         0.21000    0.00020   100%  1097 14872s
 40217 30939    0.18705  323 6014    0.21000    0.00020   100%  1109 15093s
 40812 31509

H96260 68205                       0.0800000    0.00020   100%  1577 47371s
 96376 68286 infeasible  272         0.08000    0.00020   100%  1581 47720s
 97272 68888    0.02107  299 5237    0.08000    0.00020   100%  1586 48080s
 97708 69190    0.01004  389 5799    0.08000    0.00020   100%  1591 48454s
 98585 69960    0.01004  408 5802    0.08000    0.00020   100%  1596 49039s
 99165 70495    0.01001  484 5337    0.08000    0.00020   100%  1606 49429s
 100145 71221    0.01001  598 5008    0.08000    0.00020   100%  1611 49893s
 100381 71484    0.01001  599 4988    0.08000    0.00020   100%  1611 50273s
 101337 72228    0.01001  727 6064    0.08000    0.00020   100%  1618 50671s
 101881 72738 infeasible  872         0.08000    0.00020   100%  1626 51040s
 102721 73550    0.01004  283 4893    0.08000    0.00020   100%  1638 51555s
 103519 74317    0.01004  282 4188    0.08000    0.00020   100%  1643 51908s
 104453 75182    0.00020  242 3084    0.08000    0.00020   100%  1649 52525s
H1049

 138282 53574    0.00020  311 1438    0.02000    0.00020  99.0%  1645 67438s
 139145 54048    0.00020  247 1135    0.02000    0.00020  99.0%  1642 67538s
 139839 54477    0.00020  278 1524    0.02000    0.00020  99.0%  1641 67641s
 140454 54851    0.00020  342 1578    0.02000    0.00020  99.0%  1641 67748s
 141186 55328    0.00020  385 1930    0.02000    0.00020  99.0%  1640 67851s
 141881 55728    0.00020  417 2281    0.02000    0.00020  99.0%  1640 67972s
 142838 56284    0.00020  473 1419    0.02000    0.00020  99.0%  1637 68102s
 144019 56922    0.00020  536 1801    0.02000    0.00020  99.0%  1632 68224s
 145030 57469    0.00020  208 1739    0.02000    0.00020  99.0%  1629 68349s
 145873 57970    0.00020  258 1758    0.02000    0.00020  99.0%  1627 68480s
 146758 58543 infeasible  295         0.02000    0.00020  99.0%  1627 68609s
 147725 59025    0.00020  115 1898    0.02000    0.00020  99.0%  1624 68737s
 148925 59832    0.00020  206 1635    0.02000    0.00020  99.0%  1619 68860s

...

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\Christoph\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 384, in write
    self._schedule_flush()
  File "C:\Users\Christoph\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 325, in _schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "C:\Users\Christoph\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 203, in schedule
    self._event_pipe.send(b'')
  File "zmq/backend/cython/socket.pyx", line 636, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 683, in zmq.backend.cython.socket.Socket.send
  File "zmq/backend/cython/socket.pyx", line 201, in zmq.backend.cython.socket._send_copy
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
    PyErr_CheckSignals()
KeyboardInterrupt


 336865 192247    0.00020  238 2309    0.02000    0.00020  99.0%  1355 100304s
 337485 192931    0.00020  247 2051    0.02000    0.00020  99.0%  1355 100516s
 338170 193505    0.00020  296 2629    0.02000    0.00020  99.0%  1355 100714s
 338891 194179    0.00020  196 2136    0.02000    0.00020  99.0%  1354 100715s
 338893 194148    0.00020  338 1882    0.02000    0.00020  99.0%  1355 100901s
 339379 194602    0.00020  399 1886    0.02000    0.00020  99.0%  1355 101101s
 340005 195188    0.00020  466 1675    0.02000    0.00020  99.0%  1355 101332s
 340863 196027    0.00020  516 1745    0.02000    0.00020  99.0%  1354 101546s
 341640 196761    0.00020  657 1783    0.02000    0.00020  99.0%  1353 101751s
 342491 197583    0.00020  764 1752    0.02000    0.00020  99.0%  1352 101939s
 342973 198047    0.00020   75 2507    0.02000    0.00020  99.0%  1353 102113s
 343471 198432    0.00020  106 2273    0.02000    0.00020  99.0%  1353 102333s
 344250 199221    0.00020  177 1982    0.02000    0.

 407331 259763    0.00020  324 3125    0.02000    0.00020  99.0%  1332 117769s
 407955 260281    0.00020  367 2803    0.02000    0.00020  99.0%  1332 117925s
 408507 260799    0.00020  398 2941    0.02000    0.00020  99.0%  1333 118085s
 409059 261384    0.00020  446 3076    0.02000    0.00020  99.0%  1333 118234s
 409332 261644    0.00020  187 3486    0.02000    0.00020  99.0%  1333 118235s
 409715 262005    0.00020  489 3003    0.02000    0.00020  99.0%  1333 118422s
 410343 262585    0.00020  523 2578    0.02000    0.00020  99.0%  1333 118600s
 411081 263338    0.00020  578 2513    0.02000    0.00020  99.0%  1333 118818s
 411949 264099    0.00020  640 2533    0.02000    0.00020  99.0%  1333 118963s
 412460 264594 infeasible  668         0.02000    0.00020  99.0%  1334 119122s
 413090 265184    0.00020  175 2436    0.02000    0.00020  99.0%  1334 119288s
 413739 265736    0.00020  190 1180    0.02000    0.00020  99.0%  1333 119413s
 414088 265980    0.00020  204 2123    0.02000    0.

 473659 321193    0.00020  127 3150    0.02000    0.00020  99.0%  1323 137153s
 474413 321946    0.00020  158 2862    0.02000    0.00020  99.0%  1323 137371s
 475157 322597    0.00020  219 2716    0.02000    0.00020  99.0%  1323 137579s
 475754 323196    0.00020  274 2767    0.02000    0.00020  99.0%  1323 137771s
 476301 323683    0.00020  154 2526    0.02000    0.00020  99.0%  1323 137971s
 476906 324238 infeasible  185         0.02000    0.00020  99.0%  1324 138166s
 477551 324863    0.00020  108 2751    0.02000    0.00020  99.0%  1324 138310s
 477889 325192    0.00020   86 2379    0.02000    0.00020  99.0%  1325 138467s
 478378 325650    0.00020  141 1863    0.02000    0.00020  99.0%  1325 138678s
 479088 326355    0.00020  218 2138    0.02000    0.00020  99.0%  1325 138906s
 479895 327069    0.00020  328 1999    0.02000    0.00020  99.0%  1325 139142s
 480662 327885    0.00020  445 2087    0.02000    0.00020  99.0%  1325 139414s
 481583 328785    0.00020  145 3170    0.02000    0.

In [ ]:
allvars = m_vec.getVars()

for i in allvars:
    print(str(i.varName) + " = " + str(i.x))

## needs to be in m_vec build

In [ ]:
#create result dicts for vector Model m_vec:
dic_w1 = m_vec.getAttr('x', w1)
dic_w2 = m_vec.getAttr('x', w2)
dic_w3 = m_vec.getAttr('x', w3)

dic_theta1 = m_vec.getAttr('x', theta1)
dic_theta2 = m_vec.getAttr('x', theta2)
dic_theta3 = m_vec.getAttr('x', theta3)

mydict = {
            "w1": dic_w1, 
            "w2": dic_w2, 
            "w3": dic_w3, 
            "theta1": dic_theta1, 
            "theta2": dic_theta2, 
            "theta3": dic_theta3
}

In [ ]:


with open('m_vec.txt', 'w') as f:
    print(mydict, file=f)

## Storage for Modules to test

In [ ]:
# Make crossvalidation samples

completeSamples = np.array([0,0])
for i in range(256):
    if i != 0:
        for j in range(len(npSamples)):
            if j != 0:
                completeSamples = np.vstack((completeSamples, npSamples[j]*i))

completeLabels = npLabels
addLabels = np.array([1,1,-1])
for i in range(255):
    if i !=0:
        completeLabels = np.concatenate((completeLabels,addLabels))
        
indices = np.random.permutation(completeSamples.shape[0])
training_idx, test_idx = indices[:80], indices[80:]
trainingSamples, testSamples = completeSamples[training_idx,:], completeSamples[test_idx,:]
trainingLabels = completeLabels[training_idx]
testLabels = completeLabels[test_idx]

## Crossvalidation methods

In [ ]:
def jack_valid(r_h1neurons,s_h2neurons,CVtest,CVtestL,dic_w1,dic_w2,dic_w3,dic_theta1,dic_theta2,dic_theta3):
    zeta1=np.zeros(r_h1neurons)
    for r in range(r_h1neurons):
        test1 = sum(CVtest[n] * dic_w1[n,r] for n in range(len(CVtest)))

        if test1 >= dic_theta1[r]:
            zeta1[r] = 1

    zeta2 = np.zeros(s_h2neurons)
    for s in range(s_h2neurons): 
        test2 = sum(zeta1[r]*dic_w2[r,s] for r in range(len(zeta1)))

        if test2 >= dic_theta2[s]:
            zeta2[s] = 1

    test3 = sum(zeta2[s]*dic_w3[s] for s in range(len(zeta2)))

    if test3 >= dic_theta3[0]:
        out = 1
    else:
        out = -1

    print(zeta1, zeta2, test3)
    print(out,CVtestL)
        
    if out == CVtestL:
        return True
    else:
        return False
    
def cross_valid(r_h1neurons,s_h2neurons,CVtest,CVtestL,dic_w1,dic_w2,dic_w3,dic_theta1,dic_theta2,dic_theta3):
    count = 0
    for n in range(len(CVtest)):
        zeta1=np.zeros(r_h1neurons)
        for r in range(r_h1neurons):
            test1 = sum(CVtest[n,m] * dic_w1[m,r] for m in range(len(CVtest[0])))

            if test1 >= dic_theta1[r]:
                zeta1[r] = 1

        zeta2 = np.zeros(s_h2neurons)
        for s in range(s_h2neurons): 
            test2 = sum(zeta1[r]*dic_w2[r,s] for r in range(r_h1neurons))

            if test2 >= dic_theta2[s]:
                zeta2[s] = 1

        test3 = sum(zeta2[s]*dic_w3[s] for s in range(s_h2neurons))

        if test3 >= dic_theta3[0]:
            out = 1
        else:
            out = -1

        if out == CVtestL[n]:
            count += 1
    return count/len(CVtestL)*100

In [ ]:
#Method used inside for loop before!
dic_w1, dic_w2, dic_w3, dic_theta1, dic_theta2, dic_theta3, dic_z = set_model(trainingSamples,trainingLabels,h1,h2) #Training via Leave_one_out_method

print(dic_z)

CV=0
CV=jack_valid(h1,h2,testSamples,testLabels,dic_w1,dic_w2,dic_w3,dic_theta1,dic_theta2,dic_theta3)
print(CV)

In [ ]:
##Test transformation of imported MNIST data
s=[]
for j in range(len(x[1])):
    s.append(x[1,j])
c=[]
for i in range(len(train_images[1])):
    for j in range(len(train_images[1,i])):
            c.append(train_images[1,i,j])
if len(s) == len(c):
    print("correct dimensional transformation")
else:
    print("error")
count = 0
for i in range(len(s)):
    if s[i]==c[i]:
        count+=1
    else:
        print("s: "+str(s[i])+"\nc: "+str(c[i]))
if count == 784:
    print("correct transformation")

In [ ]:
## Data evaluation of MNIST dataset!
import matplotlib.pyplot as plt

ValueCheck = [sum(x[i])/784 for i in range(60000)]
MeanValue = sum(ValueCheck)/60000 
MaxValue = max(ValueCheck)
MinValue = min(ValueCheck)

#print(ValueCheck)
print('Mean: '+ str(MeanValue))
print('Max: ' + str(MaxValue)+', Min: '+ str(MinValue))

MeanedValueCheck = [ValueCheck[i] - MeanValue for i in range(len(ValueCheck))]

sum(MeanedValueCheck)

# the histogram of the data
n, bins, patches = plt.hist(ValueCheck, 50, density=True, facecolor='g', alpha=0.75)


plt.xlabel('MeanPixelValue')
plt.ylabel('Anzahl der Inputs')
plt.axis([0, 105, 0, 0.05])
plt.grid(True)
plt.show()
np.percentile(bins,[20,50])